In [1]:
import torch
import torch.nn.functional as F

from jamba import JambaLMConfig, JambaLM, from_pretrained, load_balancing_loss

from transformers import AutoModelForCausalLM

torch.autograd.set_detect_anomaly(True)

/Users/alexandretorres/miniconda3/envs/torch23/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = from_pretrained("TechxGenus/Mini-Jamba")

model_hf = AutoModelForCausalLM.from_pretrained("TechxGenus/Mini-Jamba", torch_dtype=torch.float32, use_mamba_kernels=False, 
                                             device_map="auto", trust_remote_code=True)

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. If you want to use the naive implementation, set `use_mamba_kernels=False` in the model config


In [3]:
data = torch.randint(0, 600, (1, 20))
x = data[:, :-1]
y = data[:, 1:]

In [4]:
logits_hf = model_hf(x).logits

loss_hf = F.cross_entropy(logits_hf.view(-1, logits_hf.size(-1)), y.view(-1))
loss_hf.backward()
loss_hf

/Users/alexandretorres/miniconda3/envs/torch23/lib/python3.12/site-packages/torch/autograd/__init__.py:266: UserWarning: Error detected in MulBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/alexandretorres/miniconda3/envs/torch23/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/alexandretorres/miniconda3/envs/torch23/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/alexandretorres/miniconda3/envs/torch23/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/Users/alexandretorres/miniconda3/envs/torch23/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/Users/alexandretorres/miniconda3/envs/t

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 512, 16]], which is output 0 of torch::autograd::CopyBackwards, is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [6]:
logits, router_logits = model(x)
aux_loss = load_balancing_loss(router_logits, 8, 2)

logits_hf = model_hf(x).logits

In [7]:
torch.allclose(logits_hf, logits, atol=0.00001)

True

In [8]:
loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
loss.backward()
loss

tensor(57.9673, grad_fn=<NllLossBackward0>)

In [9]:
loss_hf = F.cross_entropy(logits_hf.view(-1, logits_hf.size(-1)), y.view(-1))
loss_hf.backward()
loss_hf

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 512, 16]], which is output 0 of torch::autograd::CopyBackwards, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [10]:
from transformers import AutoTokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained(
    "TechxGenus/Mini-Jamba",
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
tokenizer.eos_token_id

2

In [26]:
from jamba import JambaLMConfig, Jamba

In [24]:
config = JambaLMConfig(d_model=288, n_layers=8, num_attention_heads=6, num_key_value_heads=6, mlp_size=864, attn_layer_offset=8, attn_layer_period=200, expert_layer_offset=1, expert_layer_period=2, num_experts=1, num_experts_per_tok=2)
model = Jamba(config)

In [25]:
model

Jamba(
  (layers): ModuleList(
    (0-7): 8 x MambaLayer(
      (mamba): MambaBlock(
        (in_proj): Linear(in_features=288, out_features=1152, bias=False)
        (conv1d): Conv1d(576, 576, kernel_size=(4,), stride=(1,), padding=(3,), groups=576)
        (x_proj): Linear(in_features=576, out_features=50, bias=False)
        (dt_proj): Linear(in_features=18, out_features=576, bias=True)
        (out_proj): Linear(in_features=576, out_features=288, bias=False)
      )
      (moe): SparseMoEBlock(
        (experts): ModuleList(
          (0): MLP(
            (gate_proj): Linear(in_features=288, out_features=864, bias=False)
            (down_proj): Linear(in_features=864, out_features=288, bias=False)
            (up_proj): Linear(in_features=288, out_features=864, bias=False)
          )
        )
      )
      (input_layernorm): RMSNorm()
      (pre_moe_layernorm): RMSNorm()
    )
  )
)